In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(''), '../'))

from evaluation.evaluation_harness import WarmFineTuneEvaluator, include_all_users
from pprint import pprint

from dataset.data_loader import DataLoader, playtime_forever_users_games_edge_scoring_function, LinearNormalizer, GaussianNormalizer, PercentileNormalizer, never_remove_edge, remove_zero_playtime_edge
from models.random_model import RandomModel
from models.common_neighbors_model import CommonNeighbors
from models.popularity_model import GamePopularityModel
from models.ncf_model import NCFModel

In [2]:
def get_evaluator_metrics(evaluator):
    evaluator.plot_top_N_recall_percentiles(10)
    evaluator.plot_log_user_percentile_positional_error()
    evaluator.compute_mean_positional_error()
    evaluator.compute_top_N_recall(10)
    evaluator.plot_roc_curve()
    evaluator.compute_auc_roc()
    evaluator.plot_user_rank_roc_curve()
    evaluator.compute_user_rank_auc_roc()
    evaluator.plot_top_N_hit_percentage_percentiles(10)
    evaluator.compute_top_N_hit_percentage(10)
    evaluator.plot_top_N_hit_percentage_percentiles(50)
    evaluator.compute_top_N_hit_percentage(50)
    evaluator.compute_top_N_recall(50)
    evaluator.plot_top_N_recall_percentiles(50)

In [3]:
data_loader = DataLoader(cache_local_dataset=True, users_games_edge_scoring_function = playtime_forever_users_games_edge_scoring_function, user_game_edge_embeddings = ['playtime_forever'], users_games_edge_score_normalizers = [GaussianNormalizer(1.0, 1.0)], interactions_score_normalizers = [GaussianNormalizer(0.0, 1.0)])
data_loader.load_random_edge_train_tune_test_split(train_percentage=0.6, fine_tune_percentage=0.2, test_percentage=0.2, seed=0)
data_loader.save_data_loader_parameters('test_evaluation_warm_fine_tune_data_loader', overwrite=True)
evaluator = WarmFineTuneEvaluator(data_loader, top_N_games_to_eval = 500, num_users_to_eval=10000, seed=0, debug=True)

In [3]:
random_model_initializer = lambda: RandomModel()
evaluator.reset(random_model_initializer, network_save_file = 'evaluation_warm_fine_tune_test_random_model')
get_evaluator_metrics(evaluator)
evaluator.save_metrics('evaluation_warm_fine_tune_test_random_model', overwrite=True)
print(evaluator.model.name())
pprint(evaluator.metrics)
print()

Loading model: evaluation_warm_fine_tune_test_random_model
Doen loading model. evaluation_warm_fine_tune_test_random_model
Getting predictions.


User Predictions:   8%|▊         | 789/10000 [00:30<05:00, 30.70it/s]

In [ ]:
popularity_model_initializer = lambda: GamePopularityModel()
evaluator.reset(popularity_model_initializer, network_save_file = 'evaluation_warm_fine_tune_test_popularity_model')
get_evaluator_metrics(evaluator)
evaluator.save_metrics('evaluation_warm_fine_tune_test_popularity_model', overwrite=True)
print(evaluator.model.name())
pprint(evaluator.metrics)
print()

In [ ]:
common_neighbors_initializer = lambda: CommonNeighbors()
evaluator.reset(common_neighbors_initializer, network_save_file = 'evaluation_warm_fine_tune_test_common_neighbors')
get_evaluator_metrics(evaluator)
evaluator.save_metrics('evaluation_warm_fine_tune_test_common_neighbors', overwrite=True)
print(evaluator.model.name())
pprint(evaluator.metrics)
print()

In [3]:
save_file_name = 'evaluation_warm_fine_tune_test_cf'
cf_initializer = lambda: NCFModel(num_epochs = 40, embedding_size = 50, batch_percent = 0.1, learning_rate = 1e-2, weight_decay=1e-6, mlp_hidden_layer_sizes = [], seed=0, model_type='cf', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5, save_file_name=save_file_name, nn_save_name='best')
evaluator.reset(cf_initializer, network_save_file = save_file_name)
get_evaluator_metrics(evaluator)
evaluator.save_metrics(f'TEMP_{save_file_name}_200ep_1e-2_lr_1e-2_decay_batched_dropout', overwrite=True)
print(evaluator.model.name())
pprint(evaluator.metrics)
print()

Loading model: evaluation_warm_fine_tune_test_cf
Doen loading model. evaluation_warm_fine_tune_test_cf
Getting predictions.


User Predictions: 100%|██████████| 10000/10000 [4:53:30<00:00,  1.76s/it] 


Done getting predictions.
Appending dataframe information.
Ranking top N.
Done getting edge results.
neural_collborative_filtering_cf
{'auc_roc': 0.6901322410818453,
 'log_positional_error_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'mean_positional_error': 14.134260204240249,
 'roc_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage': 0.05916,
 'top_10_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage_variance': 0.0064661590615896665,
 'top_10_recall': nan,
 'top_10_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_recall_variance': nan,
 'top_50_hit_percentage': 0.02303195393609213,
 'top_50_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_hit_percentage_variance': 0.0011308080369440644,
 'top_50_recall': nan,
 'top_50_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_recall_variance': nan,
 'user_rank_auc_roc': 0.590

In [4]:
save_file_name = 'evaluation_warm_fine_tune_test_gcf'
gcf_initializer = lambda: NCFModel(num_epochs = 40, embedding_size = 50, batch_percent = 0.1, learning_rate = 1e-2, weight_decay=1e-6, mlp_hidden_layer_sizes = [], seed=0, model_type='gcf', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5, save_file_name=save_file_name, nn_save_name='best')
evaluator.reset(gcf_initializer, network_save_file = save_file_name)
get_evaluator_metrics(evaluator)
evaluator.save_metrics(f'TEMP_{save_file_name}_200ep_1e-2_lr_1e-2_decay_batched_dropout', overwrite=True)
print(evaluator.model.name())
pprint(evaluator.metrics)
print()

Loading model: evaluation_warm_fine_tune_test_gcf
Doen loading model. evaluation_warm_fine_tune_test_gcf
Getting predictions.


User Predictions:   0%|          | 0/10000 [00:00<?, ?it/s]

User Predictions: 100%|██████████| 10000/10000 [5:18:33<00:00,  1.91s/it]  


Done getting predictions.
Appending dataframe information.
Ranking top N.
Done getting edge results.
neural_collborative_filtering_gcf
{'auc_roc': 0.5677182891090459,
 'log_positional_error_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'mean_positional_error': 14.390401697770708,
 'roc_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage': 0.08648,
 'top_10_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage_variance': 0.011623325833259252,
 'top_10_recall': nan,
 'top_10_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_recall_variance': nan,
 'top_50_hit_percentage': 0.038052,
 'top_50_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_hit_percentage_variance': 0.003333971963943243,
 'top_50_recall': nan,
 'top_50_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_recall_variance': nan,
 'user_rank_auc_roc': 0.625521312356523

In [4]:
save_file_name = 'evaluation_warm_fine_tune_test_mlp'
mlp_initializer = lambda: NCFModel(num_epochs = 40, embedding_size = 50, batch_percent = 0.1, learning_rate = 1e-2, weight_decay=1e-6, mlp_hidden_layer_sizes = [128, 128, 128], seed=0, model_type='mlp', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5, save_file_name=save_file_name, nn_save_name='best')
evaluator.reset(mlp_initializer, network_save_file = save_file_name)
get_evaluator_metrics(evaluator)
evaluator.save_metrics(f'TEMP_{save_file_name}_200ep_1e-2_lr_1e-2_decay_batched_dropout', overwrite=True)
print(evaluator.model.name())
pprint(evaluator.metrics)
print()

Loading model: evaluation_warm_fine_tune_test_mlp
Doen loading model. evaluation_warm_fine_tune_test_mlp
Getting predictions.


User Predictions: 100%|██████████| 10000/10000 [5:36:32<00:00,  2.02s/it]  


Done getting predictions.
Appending dataframe information.
Ranking top N.
Done getting edge results.
neural_collborative_filtering_mlp
{'auc_roc': 0.7165619251450559,
 'log_positional_error_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'mean_positional_error': 13.32694751362681,
 'roc_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage': 0.03952,
 'top_10_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage_variance': 0.0038382079820788824,
 'top_10_recall': nan,
 'top_10_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_recall_variance': nan,
 'top_50_hit_percentage': 0.020263837889296886,
 'top_50_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_hit_percentage_variance': 0.0008162486915191787,
 'top_50_recall': nan,
 'top_50_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_recall_variance': nan,
 'user_rank_auc_roc': 0.60

In [5]:
save_file_name = 'evaluation_warm_fine_tune_test_ncf'
ncf_initializer = lambda: NCFModel(num_epochs = 40, embedding_size = 25, batch_percent = 0.1, learning_rate = 1e-2, weight_decay=1e-6, mlp_hidden_layer_sizes = [128, 128, 128], seed=0, model_type='ncf', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5, save_file_name=save_file_name, nn_save_name='best')
evaluator.reset(ncf_initializer, network_save_file = save_file_name)
get_evaluator_metrics(evaluator)
evaluator.save_metrics(f'TEMP_{save_file_name}_200ep_1e-2_lr_1e-2_decay_batched_dropout', overwrite=True)
print(evaluator.model.name())
pprint(evaluator.metrics)
print()

Loading model: evaluation_warm_fine_tune_test_ncf
Doen loading model. evaluation_warm_fine_tune_test_ncf
Getting predictions.


User Predictions: 100%|██████████| 10000/10000 [8:13:18<00:00,  2.96s/it]  


Done getting predictions.
Appending dataframe information.
Ranking top N.
Done getting edge results.
neural_collborative_filtering_ncf
{'auc_roc': 0.7055878051649971,
 'log_positional_error_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'mean_positional_error': 13.38606673622244,
 'roc_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage': 0.04688,
 'top_10_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage_variance': 0.004608311683116386,
 'top_10_recall': nan,
 'top_10_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_recall_variance': nan,
 'top_50_hit_percentage': 0.025246,
 'top_50_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_hit_percentage_variance': 0.0010101215042432135,
 'top_50_recall': nan,
 'top_50_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_recall_variance': nan,
 'user_rank_auc_roc': 0.602202281065615

In [3]:
data_loader_embeddings = DataLoader(cache_local_dataset=True, users_games_edge_scoring_function = playtime_forever_users_games_edge_scoring_function, user_game_edge_embeddings = ['playtime_forever'], users_games_edge_score_normalizers = [GaussianNormalizer(1.0, 1.0)], interactions_score_normalizers = [GaussianNormalizer(0.0, 1.0)], game_embeddings=['numReviews', 'avgReviewScore', 'price', 'numFollowers'])
data_loader_embeddings.load_random_edge_train_tune_test_split(train_percentage=0.6, fine_tune_percentage=0.2, test_percentage=0.2, seed=0)
data_loader_embeddings.save_data_loader_parameters('test_evaluation_warm_fine_tune_data_loader_embed_all_except_tags_genres', overwrite=True)
evaluator_embeddings = WarmFineTuneEvaluator(data_loader_embeddings, top_N_games_to_eval = 500, num_users_to_eval=10000, user_eval_include_function=include_all_users, seed=0, debug=True)

In [4]:
save_file_name = 'evaluation_warm_fine_tune_test_cf_embed_all_except_tags_genres'
cf_initializer = lambda: NCFModel(num_epochs = 40, embedding_size = 48, batch_percent = 0.1, learning_rate = 1e-2, weight_decay=1e-6, mlp_hidden_layer_sizes = [], seed=0, model_type='cf', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5, save_file_name=save_file_name, nn_save_name='best')
evaluator_embeddings.reset(cf_initializer, network_save_file = save_file_name)
get_evaluator_metrics(evaluator_embeddings)
evaluator_embeddings.save_metrics(f'{save_file_name}', overwrite=True)
print(evaluator_embeddings.model.name())
pprint(evaluator_embeddings.metrics)
print()

Loading model: evaluation_warm_fine_tune_test_cf_embed_all_except_tags_genres
Doen loading model. evaluation_warm_fine_tune_test_cf_embed_all_except_tags_genres
Getting predictions.


User Predictions: 100%|██████████| 10000/10000 [5:20:05<00:00,  1.92s/it] 


Done getting predictions.
Appending dataframe information.
Ranking top N.
Done getting edge results.
neural_collborative_filtering_cf
{'auc_roc': 0.6960919414769773,
 'log_positional_error_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'mean_positional_error': 14.522771972380806,
 'roc_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage': 0.05734942650573494,
 'top_10_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage_variance': 0.0061680103898970415,
 'top_10_recall': nan,
 'top_10_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_recall_variance': nan,
 'top_50_hit_percentage': 0.023996,
 'top_50_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_hit_percentage_variance': 0.0011188342216684358,
 'top_50_recall': nan,
 'top_50_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_recall_variance': nan,
 'user_rank_auc_roc': 0.59

In [4]:
save_file_name = 'evaluation_warm_fine_tune_test_gcf_embed_all_except_tags_genres'
gcf_initializer = lambda: NCFModel(num_epochs = 40, embedding_size = 48, batch_percent = 0.1, learning_rate = 1e-2, weight_decay=1e-6, mlp_hidden_layer_sizes = [], seed=0, model_type='gcf', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5, save_file_name=save_file_name, nn_save_name='best')
evaluator_embeddings.reset(gcf_initializer, network_save_file = save_file_name)
get_evaluator_metrics(evaluator_embeddings)
evaluator_embeddings.save_metrics(f'{save_file_name}', overwrite=True)
print(evaluator_embeddings.model.name())
pprint(evaluator_embeddings.metrics)
print()

Loading model: evaluation_warm_fine_tune_test_gcf_embed_all_except_tags_genres
Doen loading model. evaluation_warm_fine_tune_test_gcf_embed_all_except_tags_genres
Getting predictions.


User Predictions: 100%|██████████| 10000/10000 [5:22:08<00:00,  1.93s/it] 


Done getting predictions.
Appending dataframe information.
Ranking top N.
Done getting edge results.
neural_collborative_filtering_gcf
{'auc_roc': 0.5295977789963715,
 'log_positional_error_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'mean_positional_error': 14.502239736263615,
 'roc_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage': 0.07718922810771893,
 'top_10_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage_variance': 0.011902763482362914,
 'top_10_recall': nan,
 'top_10_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_recall_variance': nan,
 'top_50_hit_percentage': 0.03482593034813931,
 'top_50_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_hit_percentage_variance': 0.0037907121496943744,
 'top_50_recall': nan,
 'top_50_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_recall_variance': nan,
 'user_rank_auc

In [4]:
save_file_name = 'evaluation_warm_fine_tune_test_mlp_embed_all_except_tags_genres'
mlp_initializer = lambda: NCFModel(num_epochs = 40, embedding_size = 50, batch_percent = 0.1, learning_rate = 1e-2, weight_decay=1e-6, mlp_hidden_layer_sizes = [128, 128, 128], seed=0, model_type='mlp', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5, save_file_name=save_file_name, nn_save_name='best')
evaluator_embeddings.reset(mlp_initializer, network_save_file = save_file_name)
get_evaluator_metrics(evaluator_embeddings)
evaluator_embeddings.save_metrics(f'{save_file_name}', overwrite=True)
print(evaluator_embeddings.model.name())
pprint(evaluator_embeddings.metrics)
print()

Loading model: evaluation_warm_fine_tune_test_mlp_embed_all_except_tags_genres
Doen loading model. evaluation_warm_fine_tune_test_mlp_embed_all_except_tags_genres
Getting predictions.


User Predictions: 100%|██████████| 10000/10000 [5:09:22<00:00,  1.86s/it]  


Done getting predictions.
Appending dataframe information.
Ranking top N.
Done getting edge results.
neural_collborative_filtering_mlp
{'auc_roc': 0.759961373909107,
 'log_positional_error_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'mean_positional_error': 13.175992200468722,
 'roc_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage': 0.04557,
 'top_10_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage_variance': 0.004822423324233703,
 'top_10_recall': nan,
 'top_10_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_recall_variance': nan,
 'top_50_hit_percentage': 0.021009873940756357,
 'top_50_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_hit_percentage_variance': 0.0009502971786884178,
 'top_50_recall': nan,
 'top_50_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_recall_variance': nan,
 'user_rank_auc_roc': 0.621

In [ ]:
save_file_name = 'evaluation_warm_fine_tune_test_ncf_embed_all_except_tags_genres'
ncf_initializer = lambda: NCFModel(num_epochs = 40, embedding_size = 24, batch_percent = 0.1, learning_rate = 1e-2, weight_decay=1e-6, mlp_hidden_layer_sizes = [128, 128, 128], seed=0, model_type='ncf', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5, save_file_name=save_file_name, nn_save_name='best')
evaluator_embeddings.reset(ncf_initializer, network_save_file = save_file_name)
get_evaluator_metrics(evaluator_embeddings)
evaluator_embeddings.save_metrics(f'{save_file_name}', overwrite=True)
print(evaluator_embeddings.model.name())
pprint(evaluator_embeddings.metrics)
print()